# Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---

In [1]:
import math
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from ipywidgets import interact, fixed
from moviepy.editor import VideoFileClip
from collections import deque
from IPython.display import HTML
%matplotlib inline

## Camera calibration

In [2]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.
Minv = None

# Make a list of calibration images
files = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
images =[]
for fname in files:
    #read in image
    img = mpimg.imread(fname)
    # Find the chessboard corners
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
        images.append(img)
        # Draw the corners
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints,\
                                imgpoints, img.shape[0:2], None, None)

### Distortion correction

In [3]:
def cal_undistort(img, objpoints, imgpoints):
    # Use cv2.calibrateCamera() and cv2.undistort()
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

### Color/gradient threshold functions

In [4]:
def color_gradient_threshold(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HSV color space and separate the V channel
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hsv[:,:,1]
    s_channel = hsv[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(s_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    #gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    #sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary))
    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    # scale up image
    combined_binary *= 255
    return color_binary, combined_binary

### Perspective transform

In [5]:
def perspective_transform(img):
    image_size = (img.shape[1], img.shape[0])
    points_src = [[565,460],[710,460],[1200,718],[250,718]]
    src = np.float32(points_src)
    pts = np.array(points_src, np.int32)
    pts = pts.reshape((-1,1,2))
    img_src_pts = img.copy()
    cv2.polylines(img_src_pts,[pts],True,(255,0,0), 2)

    offsetx = 330
    offsety = 0
    points_dst = [[offsetx, offsety], [image_size[0]-offsetx, offsety], [image_size[0]-offsetx, image_size[1]-offsety], [offsetx, image_size[1]-offsety]]
    dst = np.float32(points_dst)
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(img, M, image_size, flags=cv2.INTER_LINEAR)
    # region of interest
    warped[:, :int(2*warped.shape[0]/9)]=0
    warped[:, -int(2*warped.shape[0]/9):]=0
    warped_dst_pts = warped.copy()
    pts = np.array(points_dst, np.int32)
    pts = pts.reshape((-1,1,2))
    warped_dst_pts = cv2.polylines(warped_dst_pts,[pts],True,(255,0,0), 2)
    return  warped, Minv, img_src_pts, warped_dst_pts

In [6]:
def pipeline_test(image):
    undistorted = cal_undistort(image, objpoints, imgpoints)
    img = undistorted.copy()
    color_binary, combined_binary = color_gradient_threshold(img)
    img = cv2.cvtColor(combined_binary, cv2.COLOR_GRAY2BGR)
    binary_warped, Minv, img_src_pts, warped_dst_pts = perspective_transform(img)
    return undistorted, binary_warped, Minv, img_src_pts, warped_dst_pts

def warp_thresh(image):
    global Minv
    undistorted, binary_warped, Minv, _, _ = pipeline_test(image)
    return undistorted, binary_warped

def pipeline(image):
    global Minv
    undistorted, binary_warped, Minv, _, _ = pipeline_test(image)
    return binary_warped

## Detect lane lines

### Line class

In [7]:
n = 15 # number of historic data cells
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = deque([], maxlen = n)
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.recent_fits = deque([], maxlen = n)
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None

### histogram

In [8]:
left = Line()
right = Line()
max_distance = 2.8
max_rel_fitx = 0.15
n_avg = 8
# Define conversions in x and y from pixels space to meters
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension


def find_lanes(binary_warped, nwindows = 9, margin = 100, minpix = 50):
    binary_warped = cv2.cvtColor(binary_warped, cv2.COLOR_BGR2GRAY)
    image_size = (img.shape[1], img.shape[0])
    # Assuming you have created a warped binary image called "binary_warped"
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[int(binary_warped.shape[0]/2):,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    #nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and5 left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
        
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    y_eval = np.max(ploty)
    
    # update right lane
    if len(right.recent_fits) == 0:
        right.detected = True
        right.recent_xfitted.append(right_fitx)
        right.recent_fits.append(right_fit)
        right.current_fit = right_fit
        right.best_fit = right_fit
        right.bestx = right_fitx
    else:
        right.line_base_pos = (right_fit[0]*y_eval**2 + right_fit[1]*y_eval \
                               + right_fit[2] - 640.0)*3.7/600.0
        right.diffs = right_fitx - right.bestx
        rel_diff = np.linalg.norm(right.diffs)/np.linalg.norm(right.bestx)
        if abs(right.line_base_pos) <= max_distance \
                    and rel_diff < max_rel_fitx :
            right.detected = True
            right.recent_xfitted.append(right_fitx)
            right.bestx = np.average(np.asarray(right.recent_xfitted), 0)
            right.recent_fits.append(right_fit)
            right.best_fit = np.average(np.array(right.recent_fits), 0)
            right.current_fit = right_fit
        else:
            right.detected = False
            right.recent_fits.pop()
            right.recent_xfitted.pop()
            if len(right.recent_fits) > 0:
                right.bestx = np.average(np.asarray(right.recent_xfitted)[-n_avg:], 0)
                right.best_fit = np.average(np.array(right.recent_fits)[-n_avg:], 0)
                right.current_fit = right.best_fit
        right.allx = rightx
        right.ally = righty
        right_fit = right.best_fit
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2] 
            
    # update left lane
    if len(left.recent_fits) == 0:
        left.detected = True
        left.recent_xfitted.append(left_fitx)
        left.recent_fits.append(left_fit)
        left.current_fit = left_fit
        left.best_fit = left_fit
        left.bestx = left_fitx
    else:
        left.line_base_pos = (left_fit[0]*720.0**2 + left_fit[1]*720.0 \
                               + left_fit[2] - 640.0)*3.7/600.0
        left.diffs = left_fitx - left.bestx
        rel_diff = np.linalg.norm(left.diffs)/np.linalg.norm(left.bestx)
        if abs(left.line_base_pos) <= max_distance \
                    and rel_diff < max_rel_fitx :
            left.detected = True
            left.recent_xfitted.append(left_fitx)
            left.bestx = np.average(np.asarray(left.recent_xfitted), 0)
            left.recent_fits.append(left_fit)
            left.best_fit = np.average(np.array(left.recent_fits), 0)
            left.current_fit = left_fit
        else:
            left.detected = False
            left.recent_fits.pop()
            left.recent_xfitted.pop()
            if len(left.recent_fits) > 0:
                left.bestx = np.average(np.asarray(left.recent_xfitted)[-n_avg:], 0)
                left.best_fit = np.average(np.array(left.recent_fits)[-n_avg:], 0)
                left.current_fit = left.best_fit
        left.allx = leftx
        left.ally = lefty
        left_fit = left.best_fit
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2] 
                        
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]]\
        = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]]\
        = [0, 0, 255]
        
    warped_lanes = np.zeros_like(out_img, dtype = np.uint8)
    l_points = np.array([left_fitx, ploty], dtype = np.int32).transpose()
    r_points = np.flipud(np.array([right_fitx, ploty], dtype = np.int32).transpose())
    points = np.hstack(([l_points], [r_points]))
    color_fit_lines = np.dstack((binary_warped, binary_warped, binary_warped))*255
    cv2.polylines(color_fit_lines, [l_points], 0, (255, 0, 0), 4)
    cv2.polylines(color_fit_lines, [r_points], 0, (0, 0, 255), 4)
    cv2.fillPoly(warped_lanes, points, (0, 255, 0))
    result = cv2.warpPerspective(warped_lanes, Minv, image_size, flags=cv2.INTER_LINEAR)

    #left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    #right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])

    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    left.radius_of_curvature = left_curverad
    right.radius_of_curvature = right_curverad
   
    lane_center = (left_fitx[image_size[1]-1] + right_fitx[image_size[1]-1])/2.0
    image_center = 1280/2.0
    line_base_pos = (lane_center - image_center)*xm_per_pix
    left.line_base_pos = line_base_pos
    right.line_base_pos = line_base_pos
    str1 = str('Left radius of curvature: '+str(round(left_curverad/1000))+'km')
    cv2.putText(result,str1,(400,650), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3 ,cv2.LINE_AA)
    str2 = str('Right radius of curvature: '+str(round(right_curverad/1000))+'km')
    cv2.putText(result,str2,(400,700), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3, cv2.LINE_AA)
    str3 = str('Distance from center: '+"{0:.1f}".format(line_base_pos*100)+'cm')
    cv2.putText(result,str3,(400,600), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3, cv2.LINE_AA)
    return result, color_fit_lines

## Pipeline

In [9]:
def process_image(in_img):
    undistorted, binary_warped = warp_thresh(in_img)
    unwarped, color_fit_lines = find_lanes(binary_warped)
    out_img = cv2.addWeighted(undistorted, 1, unwarped, 0.3, 0)
    return out_img

### video

In [10]:
clip = VideoFileClip("project_video.mp4")
lane_clip = clip.fl_image(process_image)
lane_clip.write_videofile("project_video_result.mp4", audio = False)

[MoviePy] >>>> Building video project_video_result.mp4
[MoviePy] Writing video project_video_result.mp4


100%|█████████▉| 1260/1261 [04:20<00:00,  4.84it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_result.mp4 



In [11]:
%%HTML
<video width="640" height="360" controls>
  <source src="project_video_result.mp4" type="video/mp4">
</video>